In [1]:
import json
import pandas as pd
import ijson
from datetime import datetime
import time

In [2]:
ftp_start = 1499170800

In [3]:
ftp_end = 1499174460

In [4]:
ssh_start = 1499188140

In [5]:
ssh_end = 1499191920

In [6]:
start_time = time.time()

dict_epochtime = {}
dict_source_ip = {}
dict_dst_ip = {}
dict_source_port = {}
dict_dst_port = {}
dict_stream = {}
dict_sequence = {}
dict_eth_header = {}
dict_ip_header = {}
dict_tcp_header = {}
dict_payload = {}
dict_direction = {}
dict_b_labels ={}
dict_d_labels = {}


total_no = 0
packets = 0
with open("Tuesday.json", "rb") as f:
    for record in ijson.items(f, "item"):
        total_no += 1
        if 'eth:ethertype:ip:tcp' in record["_source"]["layers"]["frame"]["frame.protocols"]:
            packets += 1
            
            if (packets % 1000000) == 0:
                print((packets/12943381)*100,(time.time() - start_time))
                
            epoch_time = record["_source"]['layers']['frame']['frame.time_epoch']
            source_ip = record["_source"]["layers"]['ip']['ip.src']
            dst_ip = record["_source"]["layers"]['ip']['ip.dst']
            eth_raw = record["_source"]['layers']['eth_raw'][0]
            ip_raw = record["_source"]['layers']['ip_raw'][0]
            tcp_raw = record["_source"]['layers']['tcp_raw'][0]
            all_hex_raw = record["_source"]["layers"]["frame_raw"][0]
            
            all_header_raw = eth_raw + ip_raw + tcp_raw
            
            len_header = len(all_header_raw)
            
            payload_raw = all_hex_raw[len_header:]

            dict_epochtime[packets] = epoch_time
            dict_source_ip[packets] = source_ip
            dict_dst_ip[packets] = dst_ip
            dict_source_port[packets] = record["_source"]['layers']['tcp']['tcp.srcport']
            dict_dst_port[packets] = record["_source"]['layers']['tcp']['tcp.dstport']
            dict_stream[packets] = record["_source"]['layers']['tcp']['tcp.stream']
            dict_sequence[packets] = record["_source"]['layers']['tcp']['tcp.seq']
            dict_eth_header[packets] = eth_raw
            dict_ip_header[packets] = ip_raw
            dict_tcp_header[packets] = tcp_raw
            dict_payload[packets] = payload_raw



            if  float(epoch_time) >= ftp_start and float(epoch_time) <= ftp_end:
                ##ftp-patator = 6
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':

                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 6
                    dict_b_labels[packets] = 1
                elif source_ip =='192.168.10.50' and dst_ip == '172.16.0.1':
                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 6
                    dict_b_labels[packets] = 1

            elif  float(epoch_time) >= ssh_start and float(epoch_time) <= ssh_end:
                ##ssh-patator = 7
                if source_ip == '172.16.0.1' and dst_ip == '192.168.10.50':
                    dict_direction[packets]= 'fwd'
                    dict_d_labels[packets] = 7
                    dict_b_labels[packets] = 1 
                elif source_ip == '192.168.10.50' and dst_ip == '172.16.0.1':
                    dict_direction[packets]= 'bwd'
                    dict_d_labels[packets] = 7
                    dict_b_labels[packets] = 1 

            
            else:
                dict_direction[packets]= 'unknown'
                dict_d_labels[packets] = 0
                dict_b_labels[packets] = 0

                
print(time.time() - start_time)

7.7259566105641175 175.42711663246155
15.451913221128235 380.8236663341522
23.17786983169235 552.9528086185455
30.90382644225647 722.6745355129242
38.629783052820585 1010.972579240799
46.3557396633847 1180.2123758792877
54.08169627394882 1352.8534183502197
61.80765288451294 1528.7112338542938
69.53360949507706 1704.0103070735931
77.25956610564117 1897.1037826538086
2024.5633289813995


In [7]:
pd.DataFrame(dict_d_labels.values()).value_counts()

0    3815909
7     163311
6     110736
dtype: int64

In [8]:
packets

10710230

In [9]:
total_no

10710320

In [61]:
payload_byte_data = pd.DataFrame()

In [62]:
epoch_times_ = pd.DataFrame(dict_epochtime.values())

In [63]:
source_ips_ = pd.DataFrame(dict_source_ip.values())

In [64]:
dst_ips_ = pd.DataFrame(dict_dst_ip.values())

In [65]:
source_ports_ = pd.DataFrame(dict_source_port.values())

In [66]:
dst_ports_ = pd.DataFrame(dict_dst_port.values())

In [67]:
streams_ = pd.DataFrame(dict_stream.values())

In [68]:
seq_ = pd.DataFrame(dict_sequence.values())

In [69]:
eth_ = pd.DataFrame(dict_eth_header.values())

In [70]:
ip_ = pd.DataFrame(dict_ip_header.values())

In [71]:
tcp_ = pd.DataFrame(dict_tcp_header.values())

In [72]:
payload_ = pd.DataFrame(dict_payload.values())

In [73]:
direction_ = pd.DataFrame(dict_direction.values())

In [74]:
d_labels_ = pd.DataFrame(dict_d_labels.values())
b_labels_ = pd.DataFrame(dict_b_labels.values())

In [75]:
payload_byte_data['Epoch Time'] = epoch_times_

In [76]:
payload_byte_data['Source IP'] = source_ips_

In [77]:
payload_byte_data['Dst IP'] = dst_ips_

In [78]:
payload_byte_data['Source Port'] = source_ports_

In [79]:
payload_byte_data['Dst Port'] = dst_ports_

In [80]:
payload_byte_data['Stream Index'] = streams_

In [81]:
payload_byte_data['Sequence Number'] = seq_

In [82]:
payload_byte_data['Eth Header'] = eth_

In [83]:
payload_byte_data['IP Header'] = ip_

In [84]:
payload_byte_data['TCP Header'] = tcp_

In [85]:
payload_byte_data['Payload'] = payload_

In [86]:
payload_byte_data['Direction'] = direction_

In [87]:
payload_byte_data['Binary Label'] = b_labels_

In [88]:
payload_byte_data['Class Label'] = d_labels_

In [89]:
payload_byte_data

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499169224.398274000,192.168.10.5,192.168.10.3,49159,445,0,1,1866da9be37db8ac6f360a8b0800,4500002921a64000800643d0c0a80a05c0a80a03,c00701bd2b65b7c180a7fb2e501000fef8ba0000,000000000000,unknown,0.0,0.0
1,1499169224.398278000,192.168.10.5,192.168.10.3,49159,445,0,1,1866da9be37db8ac6f360a8b0800,4500002921a64000800643d0c0a80a05c0a80a03,c00701bd2b65b7c180a7fb2e501000fef8ba0000,000000000000,unknown,0.0,0.0
2,1499169224.398328000,192.168.10.3,192.168.10.5,445,49159,0,1,b8ac6f360a8b1866da9be37d0800,450000341b41400080064a2ac0a80a03c0a80a05,01bdc00780a7fb2e2b65b7c280100801f5510000010105...,,unknown,0.0,0.0
3,1499169224.398331000,192.168.10.3,192.168.10.5,445,49159,0,1,b8ac6f360a8b1866da9be37d0800,450000341b41400080064a2ac0a80a03c0a80a05,01bdc00780a7fb2e2b65b7c280100801f5510000010105...,,unknown,0.0,0.0
4,1499169250.131297000,192.168.10.5,192.168.10.3,49182,88,1,0,1866da9be37db8ac6f360a8b0800,4500003421a84000800643c3c0a80a05c0a80a03,c01e00586a277d3b000000008002200011de0000020405...,,unknown,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10710225,1499198430.986186000,192.168.10.14,172.217.10.6,62751,443,108096,1515,00c1b114eb31b8ac6f3607ee0800,4500002822934000800656a7c0a80a0eacd90a06,f51f01bb5275a67ba32958765010010141d20000,000000000000,NaN,NaN,NaN
10710226,1499198431.004716000,72.21.91.29,192.168.10.14,80,62739,108082,2389,b8ac6f3607ee00c1b114eb310800,45000034b48640003606225548155b1dc0a80a0e,0050f5132e4cad4169f409a980100124dee70000010105...,,NaN,NaN,NaN
10710227,1499198431.004717000,72.21.91.29,192.168.10.14,80,62738,108081,2389,b8ac6f3607ee00c1b114eb310800,4500003453ca40003606831148155b1dc0a80a0e,0050f5128cba474f015cd75680100124b72c0000010105...,,NaN,NaN,NaN
10710228,1499198431.066113000,217.117.75.46,192.168.10.51,443,43670,107946,206436,b8ac6f360ba800c1b114eb310800,450000345fbf40003306f885d9754b2ec0a80a33,01bbaa96e82075f7b1ac829d80100408c2df0000010108...,,NaN,NaN,NaN


In [90]:
payload_byte_data = payload_byte_data.dropna()

In [91]:
payload_byte_data = payload_byte_data.reset_index(drop = True)

In [92]:
payload_byte_data = payload_byte_data.astype({"Source Port": int, "Dst Port": int})

In [93]:
payload_byte_data.tail()

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
4089951,1499171464.556366000,13.107.4.50,192.168.10.15,80,50580,9011,657171459,001e4fd4ca2800c1b114eb310800,450005dc038f400076061f390d6b0432c0a80a0f,0050c5943afcb012c6ab5e93501003fe95ed0000,29d25be7dd2726626b7eff41d89a7db674f277ad59db65...,unknown,0.0,0.0
4089952,1499171464.556645000,13.107.4.50,192.168.10.15,80,50580,9011,657172919,001e4fd4ca2800c1b114eb310800,450005dc0390400076061f380d6b0432c0a80a0f,0050c5943afcb5c6c6ab5e93501003fee5770000,2b2b51cf9e9dc9c77d74bbbdd24ffa634b22ee46f2e8bf...,unknown,0.0,0.0
4089953,1499171464.556728000,192.168.10.15,13.107.4.50,50580,80,9011,60189,00c1b114eb31001e4fd4ca280800,450000282f3a40008006ef41c0a80a0f0d6b0432,c5940050c6ab5e933afcc6e250100675e0080000,000000000000,unknown,0.0,0.0
4089954,1499171464.556837000,13.107.4.50,192.168.10.15,80,50580,9011,657174379,001e4fd4ca2800c1b114eb310800,45000b9003914000760619830d6b0432c0a80a0f,0050c5943afcbb7ac6ab5e93501003fee7d60000,beb93f8e99d5d031b378be72e6a7cd75fb585c3d3b0db6...,unknown,0.0,0.0
4089955,1499171464.556965000,192.168.10.15,13.107.4.50,50580,80,9011,60189,00c1b114eb31001e4fd4ca280800,450000282f3b40008006ef40c0a80a0f0d6b0432,c5940050c6ab5e933afcd24a50100675d4a00000,000000000000,unknown,0.0,0.0


In [94]:
len(payload_byte_data)

4089956

In [95]:
payload_byte_data['Direction'].value_counts()

unknown    3815909
bwd         164845
fwd         109202
Name: Direction, dtype: int64

In [96]:
payload_byte_data.groupby('Direction')['Class Label'].value_counts()

Direction  Class Label
bwd        7.0              97616
           6.0              67229
fwd        7.0              65695
           6.0              43507
unknown    0.0            3815909
Name: Class Label, dtype: int64

In [97]:
df_b = payload_byte_data[payload_byte_data['Class Label']==0]

In [98]:
df_a = payload_byte_data[payload_byte_data['Class Label'] != 0]

In [99]:
df_b = df_b.reset_index(drop = True)

In [100]:
df_a = df_a.reset_index(drop = True)

In [101]:
df1 = pd.read_csv('troubleshooting dataset/Tuesday-WorkingHours.csv')

In [102]:
def df_prep_frwrd_prts(df1):
    
    df1 = df1[df1['Protocol'] ==6]
    df1 = df1[df1['Label'] =='BENIGN']
    #df1 = df1[['Src IP', 'Dst IP', 'Src Port', 'Dst Port', 'Timestamp','Label']].copy()
    
    #unique src prt and dst prt
    srcprt_list = df1['Src Port'].unique()
    dstprt_list = df1['Dst Port'].unique()
    print(len(srcprt_list))
    print('-----------')
    print(len(dstprt_list))
    print('-----------')
    intersect = set(srcprt_list).intersection(dstprt_list)
    print(len(intersect))
    print('-----------')
    print(intersect)
    
    
    df1 = df1[~df1['Src Port'].isin(intersect)]
    
    #calculate the epoch time from timestamp
    #df1['epoch']=df1['Timestamp'].apply(lambda x: (datetime.strptime(x, '%d/%m/%Y %H:%M')) if x.count(':') == 1 else  (datetime.strptime(x, '%d/%m/%Y %H:%M:%S')))
    #df1['epoch']=df1['epoch'].apply(lambda x: int((datetime(x.year,x.month,x.day,(x.hour+12),x.minute,x.second)).timestamp()) if (x.hour>=1)&(x.hour<=7) else int((datetime(x.year,x.month,x.day,x.hour,x.minute,x.second)).timestamp()))
    
    df1 = df1.reset_index(drop=True)
    
    return df1

In [103]:
df1 = df_prep_frwrd_prts(df1)

38238
-----------
588
-----------
352
-----------
{40963, 36873, 32780, 53262, 47120, 21, 22, 55322, 49181, 51241, 6187, 12334, 53304, 47161, 49233, 8275, 47192, 12376, 61531, 12386, 47205, 32875, 39033, 63612, 12416, 63623, 51338, 49302, 10395, 49309, 6303, 63684, 47306, 8396, 8399, 59600, 43218, 55516, 8417, 59624, 12521, 63737, 59641, 37115, 8443, 8444, 53508, 37127, 53516, 43287, 41244, 49439, 41251, 37157, 12585, 49451, 61743, 53562, 41276, 59712, 43344, 49494, 43360, 49509, 33126, 57718, 10621, 389, 10630, 61839, 35225, 55707, 51616, 49574, 47533, 10672, 6581, 443, 8639, 53708, 10702, 63962, 51675, 47580, 43496, 43498, 61936, 53752, 8698, 55803, 49661, 49666, 61954, 49671, 41480, 39445, 8740, 41510, 47655, 53804, 53812, 49720, 53816, 55871, 53826, 10820, 49740, 43597, 51791, 43606, 64087, 59993, 49754, 35419, 57952, 39529, 62070, 43639, 51833, 33410, 51843, 35463, 55946, 10896, 37521, 10900, 8856, 58009, 45723, 58013, 43678, 53919, 53926, 53927, 47792, 43700, 53951, 37576, 62160,

In [104]:
df_b_fwd = df_b[df_b['Source Port'].isin(df1['Src Port'].unique())] 

In [105]:
df_b_bwd = df_b[df_b['Dst Port'].isin(df1['Src Port'].unique())] 

In [106]:
df_b_fwd['Direction'] = df_b_fwd['Direction'].replace(['unknown'], 'fwd')

C:\Users\Jalal\AppData\Local\Temp\ipykernel_10980\3752041767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b_fwd['Direction'] = df_b_fwd['Direction'].replace(['unknown'], 'fwd')


In [107]:
df_b_bwd['Direction'] = df_b_bwd['Direction'].replace(['unknown'], 'bwd')

C:\Users\Jalal\AppData\Local\Temp\ipykernel_10980\3888835064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b_bwd['Direction'] = df_b_bwd['Direction'].replace(['unknown'], 'bwd')


In [108]:
df_b_bwd

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
2,1499169224.398328000,192.168.10.3,192.168.10.5,445,49159,0,1,b8ac6f360a8b1866da9be37d0800,450000341b41400080064a2ac0a80a03c0a80a05,01bdc00780a7fb2e2b65b7c280100801f5510000010105...,,bwd,0.0,0.0
3,1499169224.398331000,192.168.10.3,192.168.10.5,445,49159,0,1,b8ac6f360a8b1866da9be37d0800,450000341b41400080064a2ac0a80a03c0a80a05,01bdc00780a7fb2e2b65b7c280100801f5510000010105...,,bwd,0.0,0.0
6,1499169250.131351000,192.168.10.3,192.168.10.5,88,49182,1,0,b8ac6f360a8b1866da9be37d0800,450000341b43400080064a28c0a80a03c0a80a05,0058c01ed08a8c836a277d3c80122000b4be0000020405...,,bwd,0.0,0.0
7,1499169250.131355000,192.168.10.3,192.168.10.5,88,49182,1,0,b8ac6f360a8b1866da9be37d0800,450000341b43400080064a28c0a80a03c0a80a05,0058c01ed08a8c836a277d3c80122000b4be0000020405...,,bwd,0.0,0.0
12,1499169250.131885000,192.168.10.3,192.168.10.5,88,49182,1,1,b8ac6f360a8b1866da9be37d0800,450000db1b44400080064980c0a80a03c0a80a05,0058c01ed08a8c846a277e1850180805b7d10000,000000af7e81ac3081a9a003020105a10302011ea41118...,bwd,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815900,1499171464.556106000,192.168.10.50,172.16.0.1,21,53654,10076,55,00c1b114eb310019b90a69f10800,4500003414d740004006af01c0a80a32ac100001,0015d19620052f8459de7112801000e38e2f0000010108...,,bwd,0.0,0.0
3815903,1499171464.556259000,13.107.4.50,192.168.10.15,80,50580,9011,657167079,001e4fd4ca2800c1b114eb310800,45001144038c4000760613d40d6b0432c0a80a0f,0050c5943afc9ef6c6ab5e93501003feed8a0000,4c3b16f75ac96e7a8feca0e6cf2fa31bbf97915f49cbe4...,bwd,0.0,0.0
3815904,1499171464.556366000,13.107.4.50,192.168.10.15,80,50580,9011,657171459,001e4fd4ca2800c1b114eb310800,450005dc038f400076061f390d6b0432c0a80a0f,0050c5943afcb012c6ab5e93501003fe95ed0000,29d25be7dd2726626b7eff41d89a7db674f277ad59db65...,bwd,0.0,0.0
3815905,1499171464.556645000,13.107.4.50,192.168.10.15,80,50580,9011,657172919,001e4fd4ca2800c1b114eb310800,450005dc0390400076061f380d6b0432c0a80a0f,0050c5943afcb5c6c6ab5e93501003fee5770000,2b2b51cf9e9dc9c77d74bbbdd24ffa634b22ee46f2e8bf...,bwd,0.0,0.0


In [109]:
df_all = pd.concat([df_a, df_b_fwd , df_b_bwd])

In [110]:
df_all = df_all.reset_index(drop = True)

In [111]:
df_all

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499170800.000003000,13.107.4.50,192.168.10.14,80,50128,5612,211623539,b8ac6f3607ee00c1b114eb310800,450005dc468640007706db420d6b0432c0a80a0e,0050c3d06674f9b8d6f187ea501003ffce740000,2d04ab43650cdc5e7f37f485d64ab09fa83bb3ab0389c7...,bwd,1.0,6.0
1,1499170800.000123000,192.168.10.14,13.107.4.50,50128,80,5612,20954,00c1b114eb31b8ac6f3607ee0800,4500002874b540008006a9c7c0a80a0e0d6b0432,c3d00050d6f187ea66750520501006273ec80000,000000000000,bwd,1.0,6.0
2,1499170800.000196000,13.107.4.50,192.168.10.14,80,50128,5612,211624999,b8ac6f3607ee00c1b114eb310800,45000b90468740007706d58d0d6b0432c0a80a0e,0050c3d06674ff6cd6f187ea501003ffe7d50000,189fa2c3610086100fd908edc783be4c1cbc80e041b84c...,bwd,1.0,6.0
3,1499170800.000388000,13.107.4.50,192.168.10.14,80,50128,5612,211627919,b8ac6f3607ee00c1b114eb310800,45000b90468940007706d58b0d6b0432c0a80a0e,0050c3d066750ad4d6f187ea501003ffe7d50000,cab71d48416f5ed737efe031bae17133f7c660084eecbe...,fwd,1.0,6.0
4,1499170800.000430000,192.168.10.14,13.107.4.50,50128,80,5612,20954,00c1b114eb31b8ac6f3607ee0800,4500002874b640008006a9c6c0a80a0e0d6b0432,c3d00050d6f187ea6675163c501006272dac0000,000000000000,bwd,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072428,1499171464.556106000,192.168.10.50,172.16.0.1,21,53654,10076,55,00c1b114eb310019b90a69f10800,4500003414d740004006af01c0a80a32ac100001,0015d19620052f8459de7112801000e38e2f0000010108...,,bwd,0.0,0.0
4072429,1499171464.556259000,13.107.4.50,192.168.10.15,80,50580,9011,657167079,001e4fd4ca2800c1b114eb310800,45001144038c4000760613d40d6b0432c0a80a0f,0050c5943afc9ef6c6ab5e93501003feed8a0000,4c3b16f75ac96e7a8feca0e6cf2fa31bbf97915f49cbe4...,bwd,0.0,0.0
4072430,1499171464.556366000,13.107.4.50,192.168.10.15,80,50580,9011,657171459,001e4fd4ca2800c1b114eb310800,450005dc038f400076061f390d6b0432c0a80a0f,0050c5943afcb012c6ab5e93501003fe95ed0000,29d25be7dd2726626b7eff41d89a7db674f277ad59db65...,bwd,0.0,0.0
4072431,1499171464.556645000,13.107.4.50,192.168.10.15,80,50580,9011,657172919,001e4fd4ca2800c1b114eb310800,450005dc0390400076061f380d6b0432c0a80a0f,0050c5943afcb5c6c6ab5e93501003fee5770000,2b2b51cf9e9dc9c77d74bbbdd24ffa634b22ee46f2e8bf...,bwd,0.0,0.0


In [112]:
df_all.groupby('Direction')['Class Label'].value_counts()

Direction  Class Label
bwd        0.0            2154301
           7.0              97616
           6.0              67229
fwd        0.0            1644085
           7.0              65695
           6.0              43507
Name: Class Label, dtype: int64

In [113]:
df_all.to_pickle("Tuesday_Packet_Level_Data.pkl")